# 1. Installation

In [1]:
!pip install gradio langchain-google-genai pypdf chromadb langchain langchain_community -q
!pip install gradio langchain-google-genai pypdf chromadb langchain langchain_community -q
!pip install langchain langchain-google-genai chromadb sentence-transformers geopy folium streamlit -q
!pip install streamlit-folium -q
!pip install hf_xet -q
!pip install langchain langchain-community chromadb jq -q
!pip install langchain-huggingface -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.6 MB/s eta 0:00:

# 2. Imports & Global Variables

In [3]:
import os
import json
import gradio as gr
from typing import List, Dict, Any
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

# Set up global variables for both chatbots
resource_chatbot = None
therapist_chatbot = None
current_chatbot_type = "Therapist"  # Track current chatbot type

# 3. Therapist Chatbot System

In [4]:
THERAPIST_PROMPT = """
You are a compassionate and inclusive mental health and emotional support guide. Your role is to provide a safe, non-judgmental space for individuals to express their feelings, explore their thoughts, and receive both emotional validation and practical guidance. You are here to support people through a wide range of mental health challenges, from everyday stress to more complex emotional struggles.

[Previous therapist prompt content remains the same...]
"""

class TherapistChatbot:
    def __init__(self, google_api_key: str):
        """Initialize the therapist chatbot"""
        os.environ["GOOGLE_API_KEY"] = google_api_key

        # Initialize LLM
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            temperature=0.3,
            max_tokens=None,
            timeout=None,
            max_retries=2,
        )

        # Initialize memory
        self.memory = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True,
            output_key='answer'
        )

        # Load PDF and create splits
        self.pdf_path = "/content/Mental Wellness and Support.pdf"
        self.vectorstore = self._setup_knowledge_base()
        self.qa_chain = self._create_conversation_chain()

    def _load_pdf(self, pdf_path):
        """Load PDF document"""
        loader = PyPDFLoader(pdf_path)
        documents = loader.load()
        return documents

    def _setup_knowledge_base(self):
        """Create vector store from documents"""
        documents = self._load_pdf(self.pdf_path)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        splits = text_splitter.split_documents(documents)

        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        vectorstore = Chroma.from_documents(
            documents=splits,
            embedding=embeddings,
            persist_directory="/tmp/therapist_chroma_db"
        )
        return vectorstore

    def _create_conversation_chain(self):
        """Create the conversation chain with memory and custom prompt"""
        prompt_template = THERAPIST_PROMPT + """
        Chat History:
        {chat_history}

        Context:
        {context}

        Question: {question}

        Answer:
        """

        prompt = PromptTemplate(
            template=prompt_template,
            input_variables=["chat_history", "context", "question"]
        )

        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3}),
            memory=self.memory,
            combine_docs_chain_kwargs={"prompt": prompt},
            return_source_documents=False,
            verbose=True
        )
        return qa_chain

    def respond(self, message: str, history: list) -> str:
        """Process user message and return response"""
        try:
            response = self.qa_chain({"question": message})
            return response["answer"]
        except Exception as e:
            return f"Error processing your message: {str(e)}"

# 4. Resource Location Chatbot

In [5]:
class IndyResourceChatbot:
    """Comprehensive Indianapolis Resource Locator Chatbot"""

    def __init__(self, google_api_key: str):
        """Initialize the chatbot with Google API key"""
        os.environ["GOOGLE_API_KEY"] = google_api_key

        # Initialize LLM
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-2.0-flash-exp",
            temperature=0.3,
            max_tokens=None,
            timeout=None,
            max_retries=2,
        )

        # Initialize embedding function
        self.embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

        # Initialize the RAG system
        self.chain = None
        self.vectorstore = None

    def load_emergency_shelters(self) -> List[Document]:
        """Load and process emergency shelter data"""
        documents = []

        try:
            with open("./Human_emergency_shelters_indianapolis.json", "r") as f:
                shelters_data = json.load(f)

            for shelter in shelters_data:
                # Main shelter info
                main_content = f"""
                EMERGENCY SHELTER: {shelter['name']}
                ADDRESS: {shelter['address']}
                PHONE: {shelter['phone']}
                WEBSITE: {shelter.get('website', 'Not available')}
                SERVICES: {', '.join(shelter['services'])}
                NOTES: {shelter['notes']}
                TYPE: Human Emergency Shelter
                """

                documents.append(Document(
                    page_content=main_content.strip(),
                    metadata={
                        "type": "emergency_shelter",
                        "name": shelter["name"],
                        "address": shelter["address"],
                        "phone": shelter["phone"],
                        "services": ', '.join(shelter['services']),
                        "category": "human_services"
                    }
                ))

                # Individual service documents
                for service in shelter["services"]:
                    service_content = f"""
                    SERVICE: {service}
                    PROVIDED BY: {shelter['name']} Emergency Shelter
                    LOCATION: {shelter['address']}
                    CONTACT: {shelter['phone']}
                    ADDITIONAL INFO: {shelter['notes']}
                    """

                    documents.append(Document(
                        page_content=service_content.strip(),
                        metadata={
                            "type": "shelter_service",
                            "service": service,
                            "shelter_name": shelter["name"],
                            "category": "human_services"
                        }
                    ))
        except FileNotFoundError:
            print("Warning: Human emergency shelters file not found")

        return documents

    def load_animal_shelters(self) -> List[Document]:
        """Load and process animal shelter data"""
        documents = []

        try:
            with open("./animal_shelters_indianapolis.json", "r") as f:
                shelters_data = json.load(f)

            for shelter in shelters_data:
                contact_parts = []
                if shelter.get('phone'):
                    contact_parts.append(f"Phone: {shelter['phone']}")
                if shelter.get('email'):
                    contact_parts.append(f"Email: {shelter['email']}")
                if shelter.get('hours'):
                    contact_parts.append(f"Hours: {shelter['hours']}")

                contact_info = ", ".join(contact_parts) if contact_parts else "Contact info not available"

                main_content = f"""
                ANIMAL SHELTER: {shelter['name']}
                ADDRESS: {shelter['address']}
                {contact_info}
                WEBSITE: {shelter.get('website', 'Not available')}
                SERVICES: {', '.join(shelter['services'])}
                TYPE: Animal Shelter
                """

                documents.append(Document(
                    page_content=main_content.strip(),
                    metadata={
                        "type": "animal_shelter",
                        "name": shelter["name"],
                        "address": shelter["address"],
                        "phone": shelter.get("phone", ""),
                        "services": ', '.join(shelter['services']),
                        "category": "animal_services"
                    }
                ))

                # Individual service documents
                for service in shelter["services"]:
                    service_content = f"""
                    ANIMAL SERVICE: {service}
                    PROVIDED BY: {shelter['name']} Animal Shelter
                    LOCATION: {shelter['address']}
                    CONTACT: {shelter.get('phone', shelter.get('email', 'Contact via website'))}
                    HOURS: {shelter.get('hours', 'Contact for hours')}
                    """

                    documents.append(Document(
                        page_content=service_content.strip(),
                        metadata={
                            "type": "animal_service",
                            "service": service,
                            "shelter_name": shelter["name"],
                            "category": "animal_services"
                        }
                    ))
        except FileNotFoundError:
            print("Warning: Animal shelters file not found")

        return documents

    def load_healthcare_facilities(self) -> List[Document]:
        """Load and process healthcare facility data including mental wellness"""
        documents = []

        try:
            with open("./Hospitals_and_Clinics.json", "r") as f:
                healthcare_data = json.load(f)

            # Process hospital systems
            for system in healthcare_data.get("hospital_systems", []):
                for facility in system.get("facilities", []):
                    specialties = facility.get("specialties", [])
                    specialties_text = ", ".join(specialties) if specialties else "General healthcare"

                    content = f"""
                    HEALTHCARE FACILITY: {facility['name']}
                    TYPE: {facility['type']}
                    ADDRESS: {facility['address']}
                    PHONE: {facility['phone']}
                    WEBSITE: {facility.get('website', 'Not available')}
                    SPECIALTIES: {specialties_text}
                    NOTES: {facility.get('notes', 'No additional notes')}
                    CATEGORY: Hospital/Healthcare System
                    """

                    documents.append(Document(
                        page_content=content.strip(),
                        metadata={
                            "type": "healthcare_facility",
                            "name": facility["name"],
                            "facility_type": facility["type"],
                            "specialties": specialties_text,
                            "category": "healthcare"
                        }
                    ))

            # Process clinics and urgent care
            for clinic in healthcare_data.get("clinics_and_urgent_care", []):
                specialties = clinic.get("specialties", [])
                specialties_text = ", ".join(specialties) if specialties else "General healthcare"

                content = f"""
                HEALTHCARE FACILITY: {clinic['name']}
                TYPE: {clinic['type']}
                ADDRESS: {clinic['address']}
                PHONE: {clinic['phone']}
                WEBSITE: {clinic.get('website', 'Not available')}
                SPECIALTIES: {specialties_text}
                NOTES: {clinic.get('notes', 'No additional notes')}
                CATEGORY: Clinic/Urgent Care
                """

                documents.append(Document(
                    page_content=content.strip(),
                    metadata={
                        "type": "healthcare_facility",
                        "name": clinic["name"],
                        "facility_type": clinic["type"],
                        "specialties": specialties_text,
                        "category": "healthcare"
                    }
                ))

            # Process mental wellness facilities
            for facility in healthcare_data.get("mental_wellness", []):
                services = facility.get("services", [])
                services_text = ", ".join(services) if services else "Mental health services"

                content = f"""
                MENTAL HEALTH FACILITY: {facility['name']}
                TYPE: {facility.get('type', 'Mental Health Center')}
                ADDRESS: {facility['address']}
                PHONE: {facility['phone']}
                WEBSITE: {facility.get('website', 'Not available')}
                SERVICES: {services_text}
                SPECIAL POPULATIONS: {facility.get('special_populations', 'All populations')}
                NOTES: {facility.get('notes', 'No additional notes')}
                CATEGORY: Mental Wellness
                """

                documents.append(Document(
                    page_content=content.strip(),
                    metadata={
                        "type": "mental_health",
                        "name": facility["name"],
                        "facility_type": facility.get("type", "Mental Health Center"),
                        "services": services_text,
                        "category": "mental_wellness"
                    }
                ))
        except FileNotFoundError:
            print("Warning: Healthcare facilities file not found")

        return documents

    def load_pharmacies(self) -> List[Document]:
        """Load and process pharmacy data"""
        documents = []

        try:
            with open("./pharmacies.json", "r") as f:
                pharmacy_data = json.load(f)

            for chain in pharmacy_data.get("pharmacies", []):
                chain_name = chain.get('chain_name', 'Unknown Chain')

                for location in chain.get('locations', []):
                    hours_info = location.get('hours', {})
                    if isinstance(hours_info, dict):
                        pharmacy_hours = hours_info.get('pharmacy', 'Hours not specified')
                        store_hours = hours_info.get('store', 'Store hours not specified')
                    else:
                        pharmacy_hours = str(hours_info)
                        store_hours = "Check with location"

                    services = location.get('services', [])
                    services_text = ', '.join(services) if services else 'Standard pharmacy services'

                    is_24_hour = "24 Hours" in str(hours_info) or "24 hour" in location.get('name', '').lower()

                    content = f"""
                    PHARMACY: {location.get('name', 'Unknown Location')}
                    CHAIN: {chain_name}
                    ADDRESS: {location.get('address', 'Address not available')}
                    PHONE: {location.get('phone', 'Phone not available')}
                    PHARMACY HOURS: {pharmacy_hours}
                    STORE HOURS: {store_hours}
                    SERVICES: {services_text}
                    24-HOUR SERVICE: {'Yes' if is_24_hour else 'No'}
                    NOTES: {location.get('notes', 'No additional notes')}
                    """

                    documents.append(Document(
                        page_content=content.strip(),
                        metadata={
                            "type": "pharmacy",
                            "name": location.get('name', 'Unknown Location'),
                            "chain": chain_name,
                            "is_24_hour": is_24_hour,
                            "services": services_text,
                            "category": "pharmacy"
                        }
                    ))
        except FileNotFoundError:
            print("Warning: Pharmacies file not found")

        return documents

    def load_public_safety(self) -> List[Document]:
        """Load and process public safety data"""
        documents = []

        try:
            with open("./public_safety.json", "r") as f:
                safety_data = json.load(f)

            # Police districts
            for dept in safety_data.get("public_safety", {}).get("police_departments", []):
                for district in dept.get("districts", []):
                    content = f"""
                    POLICE DISTRICT: {district.get('name', 'Unknown District')}
                    ADDRESS: {district.get('address', 'Not listed')}
                    PHONE: {district.get('phone', 'Not available')}
                    SERVICES: Police patrol, investigations, community services
                    DETAILS: {district.get('notes', 'No additional details')}
                    EMERGENCY: For emergencies, dial 911
                    """

                    documents.append(Document(
                        page_content=content.strip(),
                        metadata={
                            "type": "police_district",
                            "name": district.get('name', 'Unknown District'),
                            "category": "public_safety"
                        }
                    ))

            # Fire stations
            for dept in safety_data.get("public_safety", {}).get("fire_departments", []):
                for station in dept.get("stations", []):
                    content = f"""
                    FIRE STATION: Station {station.get('station_number', 'Unknown')}
                    NEIGHBORHOOD: {station.get('neighborhood', 'Unknown Area')}
                    ADDRESS: {station.get('address', 'Address not explicitly listed')}
                    PHONE: {station.get('phone', 'Refer to IFD Headquarters: (317) 327-6041')}
                    EQUIPMENT: {station.get('notes', 'Standard fire equipment')}
                    SERVICES: Fire suppression, emergency medical services, rescue operations
                    EMERGENCY: For emergencies, dial 911
                    """

                    documents.append(Document(
                        page_content=content.strip(),
                        metadata={
                            "type": "fire_station",
                            "station_number": station.get('station_number', 'Unknown'),
                            "neighborhood": station.get('neighborhood', 'Unknown Area'),
                            "category": "public_safety"
                        }
                    ))
        except FileNotFoundError:
            print("Warning: Public safety file not found")

        # Add emergency contact information
        emergency_info = """
        EMERGENCY CONTACTS INDIANAPOLIS
        POLICE EMERGENCY: 911
        FIRE EMERGENCY: 911
        MEDICAL EMERGENCY: 911
        NON-EMERGENCY POLICE: (317) 327-3811
        IMPD HEADQUARTERS: (317) 327-3811
        IFD HEADQUARTERS: (317) 327-6041
        POISON CONTROL: 1-800-222-1222
        MENTAL HEALTH CRISIS: 988 (Suicide & Crisis Lifeline)
        """

        documents.append(Document(
            page_content=emergency_info.strip(),
            metadata={
                "type": "emergency_contacts",
                "category": "emergency_info"
            }
        ))

        return documents

    def create_comprehensive_rag_system(self):
        """Create the comprehensive RAG system with all resource types"""
        print("Loading all resource data...")

        # Collect all documents
        all_documents = []
        all_documents.extend(self.load_emergency_shelters())
        all_documents.extend(self.load_animal_shelters())
        all_documents.extend(self.load_healthcare_facilities())
        all_documents.extend(self.load_pharmacies())
        all_documents.extend(self.load_public_safety())

        print(f"Loaded {len(all_documents)} documents total")

        # Create vector store
        self.vectorstore = Chroma.from_documents(all_documents, self.embedding_function)

        # Create retriever
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 8}  # Retrieve more documents for comprehensive answers
        )

        # Comprehensive prompt template
        template = """You are a helpful Indianapolis Resource Assistant that helps people find emergency services, healthcare facilities, shelters, pharmacies, and public safety resources.

Based on the following information about Indianapolis resources, answer the user's question accurately and helpfully.

Context:
{context}

Question: {question}

Instructions:
- Provide specific contact information (names, addresses, phone numbers) when available
- For emergencies, always mention "For life-threatening emergencies, dial 911"
- Include hours of operation when available
- Mention relevant services offered by each facility
- If multiple options exist, list the most relevant ones
- Be specific about which facility/service you're referring to
- If looking for specialized services, mention the specialties
- For pharmacies, include both pharmacy and store hours when different
- For shelters, mention what populations they serve (women, children, families, etc.)
- If information is not available in the context, say so clearly
- Always recommend calling ahead to verify current information

Categories I can help with:
- Emergency shelters and homeless services
- Animal shelters and pet services
- Hospitals, clinics, and healthcare facilities
- Pharmacies and medication services
- Police districts and fire stations
- Emergency contacts and crisis services
- Mental wellness and support services

Answer:"""

        prompt = ChatPromptTemplate.from_template(template)

        # Create the comprehensive RAG chain
        self.chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )

        print("Indianapolis Resource Chatbot is ready!")

    def ask(self, question: str) -> str:  # Only takes question argument
        """Main method to ask questions to the chatbot"""
        if self.chain is None:
            return "Error: Chatbot not initialized. Please run setup first."

        try:
            response = self.chain.invoke(question)
            return response
        except Exception as e:
            return f"Error processing your question: {str(e)}"



# 5. Gradio Interface

In [6]:
def initialize_chatbots(api_key):
    """Initialize both chatbots with the provided API key"""
    global resource_chatbot, therapist_chatbot

    try:
        # Initialize resource chatbot
        resource_chatbot = IndyResourceChatbot(api_key)
        resource_chatbot.create_comprehensive_rag_system()

        # Initialize therapist chatbot
        therapist_chatbot = TherapistChatbot(api_key)

        return "✅ Both chatbots initialized successfully!"
    except Exception as e:
        return f"❌ Error initializing chatbots: {str(e)}"

def chat_response(message, history, chatbot_type):
    """Process chat messages and return responses based on chatbot type"""
    global resource_chatbot, therapist_chatbot, current_chatbot_type
    current_chatbot_type = chatbot_type

    if chatbot_type == "Therapist":
        if therapist_chatbot is None:
            return "Please initialize the chatbots first by entering your Google API key in the Settings tab."
        return therapist_chatbot.respond(message, history)
    else:
        if resource_chatbot is None:
            return "Please initialize the chatbots first by entering your Google API key in the Settings tab."
        # FIXED: Only pass message to ask()
        return resource_chatbot.ask(message)
        return resource_chatbot.ask(message, history)

def get_example_questions(chatbot_type):
    """Return example questions based on chatbot type"""
    if chatbot_type == "Therapist":
        return [
            "I've been feeling really overwhelmed lately...",
            "I'm struggling with my relationships...",
            "I'm not sure how to deal with my anxiety...",
            "I've been feeling really low and don't know why...",
        ]
    else:
        return [
            "I need emergency shelter for my family with children",
            "Where can I find a 24-hour pharmacy?",
            "I found an injured stray cat, where should I take it?",
            "What hospitals have emergency rooms?",
        ]

def create_gradio_interface():
    """Create and configure the Gradio interface"""
    custom_css = """
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .emergency-notice {
        background-color: #ff4444;
        color: white;
        padding: 15px;
        border-radius: 10px;
        text-align: center;
        font-weight: bold;
        margin: 10px 0;
    }
    .resource-categories {
        background-color: #f0f8ff;
        padding: 15px;
        border-radius: 10px;
        margin: 10px 0;
    }
    .therapist-intro {
        background-color: #fff0f5;
        padding: 15px;
        border-radius: 10px;
        margin: 10px 0;
    }
    """

    with gr.Blocks(css=custom_css, title="IndyGuardian Bot") as demo:
        # Header
        gr.HTML("""
        <div style="text-align: center; padding: 20px; background: linear-gradient(90deg, #1e3a8a, #3b82f6); color: white; border-radius: 15px; margin-bottom: 20px;">
            <h1 style="margin: 0; font-size: 2.5em;">🏙️ Indianapolis Support Chatbot</h1>
            <p style="margin: 10px 0 0 0; font-size: 1.2em;">Choose between mental health support or resource finding assistance</p>
        </div>
        """)

        # Emergency notice
        gr.HTML("""
        <div class="emergency-notice">
            🚨 FOR LIFE-THREATENING EMERGENCIES, DIAL 911 IMMEDIATELY 🚨
        </div>
        """)

        # Chatbot type selection
        chatbot_type = gr.Radio(
            choices=["Therapist", "Resource Locator"],
            label="Which chatbot would you like to talk with?",
            value="Therapist"
        )

        # Dynamic description based on selection
        description = gr.HTML("""
        <div class="therapist-intro">
            <p>🌸 Hello, I'm Indianapolis Therapist. 🌸</p>
            <p>I'm here to provide a safe, non-judgmental space for you to express your feelings and receive support.</p>
        </div>
        """)

        # Update description when chatbot type changes
        def update_description(choice):
            if choice == "Therapist":
                return """
                <div class="therapist-intro">
                    <p>🌸 Hello, I'm Indianapolis Therapist. 🌸</p>
                    <p>I'm here to provide a safe, non-judgmental space for you to express your feelings, explore your thoughts, and receive support.</p>
                </div>
                """
            else:
                return """
                <div class="resource-categories">
                    <h3>📋 Available Resources:</h3>
                    <ul style="margin: 10px 0;">
                        <li>🏠 <strong>Emergency Shelters</strong> - Housing assistance and homeless services</li>
                        <li>🐕 <strong>Animal Shelters</strong> - Pet care, adoption, and veterinary services</li>
                        <li>🏥 <strong>Healthcare</strong> - Hospitals, clinics, and medical facilities</li>
                        <li>💊 <strong>Pharmacies</strong> - Medication services and 24-hour locations</li>
                        <li>👮 <strong>Public Safety</strong> - Police districts and fire stations</li>
                    </ul>
                </div>
                """

        chatbot_type.change(update_description, inputs=chatbot_type, outputs=description)


        # Chat interface
        with gr.Row():
            with gr.Column(scale=7):
                chatbot = gr.Chatbot(height=500, label="Chat History")
                msg = gr.Textbox(placeholder="Type your message here...", show_label=False)
            with gr.Column(scale=3):
                # Create separate containers for each chatbot's examples
                with gr.Column(visible=True) as therapist_examples:
                    gr.Examples(
                        examples=get_example_questions("Therapist"),
                        inputs=msg,
                        label="Therapist Examples",
                        examples_per_page=4
                    )
                with gr.Column(visible=False) as resource_examples:
                    gr.Examples(
                        examples=get_example_questions("Resource"),
                        inputs=msg,
                        label="Resource Examples",
                        examples_per_page=4
                    )

        clear_btn = gr.Button("Clear Chat")

        def respond(message, chat_history, chatbot_type):
            response = chat_response(message, chat_history, chatbot_type)
            chat_history.append((message, response))
            return "", chat_history

        msg.submit(
            respond,
            inputs=[msg, chatbot, chatbot_type],
            outputs=[msg, chatbot]
        )

        clear_btn.click(lambda: None, None, chatbot, queue=False)

        # Update examples visibility when chatbot type changes
        def update_examples_visibility(choice):
            if choice == "Therapist":
                return [gr.Column(visible=True), gr.Column(visible=False)]
            else:
                return [gr.Column(visible=False), gr.Column(visible=True)]

        chatbot_type.change(
            fn=update_examples_visibility,
            inputs=chatbot_type,
            outputs=[therapist_examples, resource_examples]
        )

        # Settings Tab
        with gr.Accordion("⚙️ Settings", open=False):
            gr.Markdown("## Chatbot Setup")
            gr.Markdown("Enter your Google API key to initialize the chatbots. You can get a free API key from [Google AI Studio](https://makersuite.google.com/app/apikey).")

            with gr.Row():
                api_key_input = gr.Textbox(
                    label="Google API Key",
                    placeholder="Enter your Google API key here...",
                    type="password",
                    scale=3
                )
                init_btn = gr.Button("Initialize Chatbots", variant="primary", scale=1)

            status_output = gr.Textbox(
                label="Status",
                interactive=False,
                max_lines=3
            )

            init_btn.click(
                fn=initialize_chatbots,
                inputs=[api_key_input],
                outputs=[status_output]
            )

    return demo



# 6. Main Execution

In [7]:
def main():
    """Main function to launch the Gradio interface"""
    print("Starting Indianapolis Support Chatbot...")

    try:
        demo = create_gradio_interface()
        demo.launch(
            server_name="0.0.0.0",
            server_port=7867,
            share=True,
            debug=True,
            show_error=True,
            inbrowser=True
        )
    except Exception as e:
        print(f"Error launching the interface: {str(e)}")

if __name__ == "__main__":
    main()

Starting Indianapolis Support Chatbot...


<ipython-input-6-2c8aac0754e2>:139: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, label="Chat History")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0b3a6f8802f843c477.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7867 <> https://0b3a6f8802f843c477.gradio.live
